In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Recurrent Neural Network (RNN) layer

**Review**

With a sequence $\x^\ip$ as input, and a sequence $\y$ as a potential output,  the questions arises:
- How does an RNN produce $\y_\tp$, the $t^{th}$ output ?

Some choices
- Predict $\y_\tp$ as a direct function of the prefix of $\x$ of length $\tt$: 
$$\pr{\y_\tp | \x_{(1)} \dots \x_\tp} $$

- Uses a "latent state" that is updated with each element of the sequence, then predict the output

$$
\begin{array}[lll] \\
\pr{\h_\tp | \x_\tp, \h_{(\tt-1)} } & \text{latent variable } \h_\tp \text{encodes } [ \x_{(1)} \dots \x_\tp ]\\
\pr{\y_\tp | \h_\tp }              & \text{prediction contingent on latent variable} \\
\end{array}
$$

In our first encounter with the RNN, we made the choice to use the "latent state" approach.

Doing so enabled us to picture an RNN as a loop:

<table>
    <tr>
        <th><center>RNN</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_loop.jpg" width=1000></td>
    </tr>
</table>

During  iteration $t$ of the loop
- We consume input $\x_\tp$
- Produce output $\y_\tp$ (which we will assume is the latent state: $\y_\tp = \h_\tp$)

We also indicated that we could "unroll" the loop

<table>
    <tr>
        <th><center>RNN unrolled</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_API_many_to_many.jpg"></td>
    </tr>
</table>

# Transformer layer

What would have happened if, rather than using the latent state approach, we choose the alternative:
- Predict $\y_\tp$ as a direct function of the prefix of $\x$ of length $\tt$:

Then the picture would look similar to the "unrolled" loop:
<table>
    <tr>
        <th><center>Transformer layer</center></th>
    </tr>
    <tr>
        <td><img src="images/Transformer_1.png"></td>
    </tr>
</table>

Compared to the unrolled RNN, the Transformer, the computation at step $t$
- Has **no** data (e.g., $\h_\tp)$ passing from the computation between time steps (from $(t-1)$, to $(t+1)$)
- Takes a **sequence** $\x_{(1..t)}$ as input
    - Because $\y_\tp$ is computed as a *direct* function of the prefix $\x_{(1..t)}$ rather than recursively

In some instances, we may even allow the Transformer to "see" the *entire* input (not just a prefix) at each step $t$
- The Encoder of an Encoder-Decoder architecture
    - Context Sensitive Encoding
        - Encode based on *entire* input
        - Bi-directional RNN

<table>
    <tr>
        <th><center>Transformer layer</center></th>
    </tr>
    <tr>
        <td><img src="images/Transformer_2.png"></td>
    </tr>
</table>

The Transformer uses *self-attention*
- To influence which elements of $\x_{(1 \ldots \, t)}$ to attend/focus to

Looking inside the circle

<table>
    <tr>
        <th><center>Transformer Layer (Encoder)</center></th>
    </tr>
    <tr>
        <td><img src="images/Transformer_Encoder.png" width=70%></td>
    </tr>
</table>

And there are cases where we *must not* allow the Transformer to "see" the *entire* input
- The Decoder of an Encoder-Decoder architecture
    - Teacher forcing: the input of step $(t+1)$ is $\y_\tp$, the output of step $t$
    - Can't look ahead to something that has not yet been created !

For those cases where look-ahead is not allowed, the Transformer using **masking**
- Hide the suffix $\y_{(t+1 \ldots \, t)}$ from the attention layer

<table>
    <tr>
        <th><center>Transformer Layer (Decoder)</center></th>
    </tr>
    <tr>
        <td><img src="images/Transformer_Decoder.png" width=70%></td>
    </tr>
</table>

You will notice two Attention layers
- **Masked** Self Attention (on $\y$)
    - Allows the layer to focus on previous outputs
        - Masked to prevent look-ahead to $\y_{(t')}$ for $t' > t$
- Encoder-Decoder Attention (on $\bar\h_\tp$)
    - Allows the Decoder to attend to the entire output sequence of the Encoder
    
So this layer attends to
- previously generated Decoder layer outputs
- the "relevant" part of the Encoder output

<table>
    <tr>
        <th><center>Transformer Layer (Encoder/Decoder)</center></th>
    </tr>
    <tr>
        <td><img src="images/Transformer_Encoder_Decoder.png" width=70%></td>
    </tr>
</table>

The Transformer architecture just stacks $N$ Transformer layers.

$N = 6$ was the choice of the original paper.

<table>
    <tr>
        <th><center>Stacked Transformer Layers (Encoder/Decoder)</center></th>
    </tr>
    <tr>
        <td><img src="images/Transformer_Encoder_Decoder_multi.png" width=70%></td>
    </tr>
</table>

## Full Encoder-Decoder Transformer architecture

<table>
    <tr>
        <th><center>Transformer (Encoder/Decoder)</center></th>
    </tr>
    <tr>
        <td><img src="images/Attention_is_all_u_need_Transformer.png" width=60%></td>
    </tr>
</table>

The diagram shows an Encoder-Decoder pair.

You will notice that each element of the pair is different.

- It is possible to use each element independently as well.

- But first we need to understand
the source of the differences and their implications.



## The Encoder

The Encoder side of the pair **does not** restrict the order in which it's inputs are accessed.
- Self-attention **without** causal masking

So the Encoder is appropriate for tasks that require a context-sensitive representation of
each input element.

For example: the meaning of the word "**it**" changes with a small change to a subsequent word in the following sentences:
- "The animal didn't cross the road because **it** was too tired"

- "The animal didn't cross the road because **it** was too wide"

Some tasks with this characteristic are
- Sentiment
- Masked Language Modeling: fill-in the masked word
- Semantic Search
    - compare a summary of the sequence that is the context-sensitive representation of
        - query sentence
        - document sentences
    - Each summary is a kind of **sentence embedding**
    - Summary
        - pooling over each word
        - final token
        

## The Decoder

One notable aspect of the Decoder is its recurrent architecture
- Output $\y_{(\tt-1)}$ is appended to the Decoder inputs available at step $\tt$.
    - The Decoder inputs are $\y_{(1..T)}$, where $T$ is the full length of the Decoder output
    - **But** Causal Masking ensures that only $\y_{(1..\tt)}$ is *available* at step $\tt$.
    
Thus, the Decoder is appropriate for *generative* tasks
- Text generation
- Predict the next word in a sentence

# Advantages of a Transformer compared to an RNN

Among the most important advantages of the Transformer over an RNN
- are its ability to capture long-term dependencies 
- because all elements of the sequence are processed in parallel
    - no vanishing gradient or truncated back propagation
    
This has made the Transformer the architecture of choice for NLP.


The computational advantages are many:
- Time: All steps computed in parallel
    - $O(1)$ sequential steps versus $O(T)$
- Fewer operations: faster training
    - $O( T^2 * d )$ versus $O(T * d^2)$, where $d$ is length of a single input element
        - e.g., $\x_\tp$ replaced by an embedding of dimension $d$
    - Transformer has fewer operations when $T \lt d$
- Similar number of parameters 
    - When $T < \sqrt{d}$: Self attention has about the same number of parameters

Note that, because of TBTT, T is the length of a *chunk* rather than the full input length
- Typical $T = 64, d \ge 256$

So under the special case (that applies to sequences) that chunk length is short relative to representation size,
it is not "crazy" to perform all elements of $\x$ with separate FC's.

The faster training enables
- larger datasets
- deeper models

## Detailed computational comparison of architectures

| Type | Parameters  | Operations\;\; | Path length |
|------|------       |------      |------       |
|  CNN | $k * d^2$   | $T * k * d^2$ | $T$ |
| RNN  | $d^2$       | $T * d^2$     | $T$ |
| Self-attention | $T^2 *d $ | $T^2 *d$ | 1 |


Here's the details of the math        

Attention involves a dot product (of vectors of length $d$)
- Each input matched against all others: $T * T$
- So $T^2 *d$ operations

RNN
- $T$ sequential steps
- Each step evaluates
    $$
\h_\tp  =  \phi(\W_{xh}\x_\tp  + \W_{hh}\h_{(t-1)}  + \b_h) 
$$
- $\h_\tp$ has multiple elements, assume $|| \h || = O(d)$
    - Computing updated hidden state element $j$ (i.e., $\h_{\tp, j}$) involves dot product of vectors of length $d$ (size of $\x_\tp)$
    - $d$ multiplications per element of $\h$, times $O(d)$ elements of $\h$ is $O(d^2)$ per step
    - So $T * d^2$ operations
    
- $\W_{hh}$ matrix: $d^2$ parameters
  - $ | \h | = d$

CNN
- path length $T$ 
    - each kernel multiplication connects only $k$ elements of $\x$
    - since kernels overlap inputs, can't parallelize, hence $O(T/k)$ path length
        - can reduce to $\log(T)$ with tree structure
- Parameters
    - kernel size $k$
    - number of input channels = number of output channels = $d$
    - $k *d$ parameters for kernel of one channel
    - $k * d^2$ parameters for kernel for all $d$ output channels
    
- Operations
    - for a single output channel: $k$ per input channel
        - There are $d$ input channels, so $k *d$ for each dot product of *one* output channel
        - There are $d$ output channels, so $k * d^2$ per time step
    - $T$ time steps so $T * k * d^2$ number of operations


RNN
- $\W_{hh}$ matrix: $d^2$ parameters
  - $ | \h | = d$
- $T * d^2$ operations (for entire sequence)
- path length $T$ 

To summarize
- for short chunk/sequence length, relative to size of hidden state
    - $|x| \lt 64$ typically; $d \approx 256$
- Transformer/self attention is comparable in terms of number of parameters

So under the special case (that applies to sequences) that chunk length is short relative to representation size,
it is not "crazy" to perform all elements of $\x$ with separate FC's.

# A free lunch ? Almost !

Transformers offer the possibility of great improvements in training speed
- Parallelism
- Fewer operations
    
Sounds too good to be true.  Is there such a thing as a free lunch ?

Almost
- RNN can handle sequences of arbitrary length ($T$ unbounded)
- Transformer has a fixed number of parallel units, which limits the length of sequences

But, in practice: RNN uses *Truncated* Back Propagation Through Time
- So the maximum distance between input sequence elements is bounded by $k$, the truncation length

## Some other advantages

- Can learn long-range dependencies
    - Gradients within a layer don't flow backwards: always a single step
        - Can't vanish or explode
    - The output $\y^{[\ll]}_\tp$ of layer $\ll$ (for stacked Transformer layers) is a function of **all** inputs
    $$
    \y^{[\ll-1]}_{(\tt')} \text{ for } 1 \le \tt' \le T
    $$
        - so can directly access a distant input
        - not diminished by passing through multiple intermediate time steps


## Some drawbacks

- The output $\y^{[\ll]}_\tp$ of layer $\ll$ (for stacked Transformer layers) is a function of **all** inputs, **always**
    - Perhaps less efficient
- Unless you add positional encoding, you lose ordering relationships between inputs

In [ ]:
print("Done")